<a href="https://colab.research.google.com/github/mohannishant6/thesis/blob/master/notebooks/traffic_violations_lgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install catboost
!pip install category_encoders
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from category_encoders import *
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score, f1_score, jaccard_score
import lightgbm as lgbm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np

ModuleNotFoundError: ignored

In [2]:
df=pd.read_csv('https://data.montgomerycountymd.gov/api/views/4mse-ku6q/rows.csv?accessType=DOWNLOAD')
# df.to_csv('data')
# df=pd.read_csv('data')


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,20,21,22,23,24,25,34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
#solving problem of unseen classes in label encoding
#https://stackoverflow.com/questions/21057621/sklearn-labelencoder-with-never-seen-before-values

class LabelEncoderExt(object):
    def __init__(self):
        """
        It differs from LabelEncoder by handling new classes and providing a value for it [Unknown]
        Unknown will be added in fit and transform will take care of new item. It gives unknown class id
        """
        self.label_encoder = LabelEncoder()
        # self.classes_ = self.label_encoder.classes_

    def fit(self, data_list):
        """
        This will fit the encoder for all the unique values and introduce unknown value
        :param data_list: A list of string
        :return: self
        """
        self.label_encoder = self.label_encoder.fit(list(data_list) + ['Unknown'])
        self.classes_ = self.label_encoder.classes_

        return self

    def transform(self, data_list):
        """
        This will transform the data_list to id list where the new values get assigned to Unknown class
        :param data_list:
        :return:
        """
        new_data_list = list(data_list)
        for unique_item in np.unique(data_list):
            if unique_item not in self.label_encoder.classes_:
                new_data_list = ['Unknown' if x==unique_item else x for x in new_data_list]

        return self.label_encoder.transform(new_data_list)

In [24]:
df1=df.loc[:,[ 'Description','Belts','Property Damage','Fatal','Commercial License','HAZMAT','Commercial Vehicle','Alcohol','Work Zone','Year','Race','Gender','Arrest Type','Violation Type']].dropna()

X=df1.iloc[:,:-1]
y=df1.iloc[:,-1]

#split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.02, random_state=42)

for col in X.columns:
  le = LabelEncoderExt()
  le.fit(X_train[col])
  X_train[col]=le.transform(X_train[col])
  X_test[col]=le.transform(X_test[col])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [26]:
#lgbm

model_lgbm = lgbm.LGBMClassifier(boosting_type='goss',objective='multiclass')
parameters = {'depth'         : [6,8,10,12,14,16],
              'learning_rate' : [.01,.05,0.1,.2],
              'iterations'    : [100, 500,1000,2000,3000]
              }
grid = GridSearchCV(estimator=model_lgbm, param_grid = parameters)
grid.fit(X_train, y_train,categorical_feature=['Description','Belts','Property Damage','Fatal','Commercial License','HAZMAT','Commercial Vehicle','Alcohol','Work Zone','Race','Gender','Arrest Type'])    

# Results from Grid Search
print("\n========================================================")
print(" Results from Grid Search " )
print("========================================================")    

print("\n The best estimator across ALL searched params:\n",
      grid.best_estimator_)

print("\n The best score across ALL searched params:\n",
      grid.best_score_)

print("\n The best parameters across ALL searched params:\n",
      grid.best_params_)

print("\n ========================================================")

c=grid.best_estimator_
print("training")
print("macro f1: ",f1_score(y_train,c.predict(X_train), average='macro'))
print("micro f1: ",f1_score(y_train, c.predict(X_train), average='micro'))
print("macro jaccard: ",jaccard_score(y_train,c.predict(X_train), average='macro'))
print("micro jaccard: ",jaccard_score(y_train, c.predict(X_train), average='micro'))

print("test")
print("macro f1: ",f1_score(y_train,c.predict(X_train), average='macro'))
print("micro f1: ",f1_score(y_train, c.predict(X_train), average='micro'))
print("macro jaccard: ",jaccard_score(y_train,c.predict(X_train), average='macro'))
print("micro jaccard: ",jaccard_score(y_train, c.predict(X_train), average='micro'))

print(multilabel_confusion_matrix(y_test, c.predict(X_test)))

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Alcohol', 'Arrest Type', 'Belts', 'Commercial License', 'Commercial Vehicle', 'Description', 'Fatal', 'Gender', 'HAZMAT', 'Property Damage', 'Race', 'Work Zone']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))



 Results from Grid Search 

 The best estimator across ALL searched params:
 LGBMClassifier(boosting_type='goss', class_weight=None, colsample_bytree=1.0,
               depth=6, importance_type='split', iterations=10,
               learning_rate=0.1, max_depth=-1, min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_leaves=31, objective='multiclass',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

 The best score across ALL searched params:
 0.7940899747117042

 The best parameters across ALL searched params:
 {'depth': 6, 'iterations': 10, 'learning_rate': 0.1}

training
macro f1:  0.6460103887241017
micro f1:  0.7955765454897418


NameError: ignored